In [48]:
from morphcloud.api import MorphCloudClient
import os

In [49]:
client = MorphCloudClient()

In [90]:
# initialize base instance
snapshot = client.snapshots.create(
    vcpus=1,
    memory=2048,
    disk_size=8192//2
)

In [ ]:
# do these chained executions for setup, then create an instance to sftp and upload files, then create another snapshot?
# snapshot = (
#     snapshot.setup("apt update -y")
#     .setup("apt-get install -y python3-pip")
#     .setup("curl -LsSf https://astral.sh/uv/install.sh | bash")
#     .setup("cd app")
#     .setup("uv add pyprpoject.toml")
# )

In [91]:
snapshot_id = snapshot.id

# uploading stuff to a base instance and branching from there

In [92]:
instance = client.instances.start(snapshot_id=snapshot_id) # brand new instance costing $$$!
base_instance_id = instance.id

In [93]:
print(f"Base snapshot id: {snapshot_id}, base instance id: {base_instance_id}")

Base snapshot id: snapshot_664am2xe, base instance id: morphvm_k71tpchf


In [94]:
result = instance.exec(command="mkdir app")

In [95]:
# upload project to base instance
with instance.ssh() as ssh:
    sftp = ssh._client.open_sftp()

    sftp.put("/home/asusevski/morph/chess/config.yaml", "app/config.yaml")
    sftp.put("/home/asusevski/morph/chess/pyproject.toml", "app/pyproject.toml")
    sftp.put("/home/asusevski/morph/chess/requirements.txt", "app/requirements.txt")
    sftp.put("/home/asusevski/morph/chess/schemas.py", "app/schemas.py")
    sftp.put("/home/asusevski/morph/chess/chess_game.py", "app/chess_game.py")
    sftp.put("/home/asusevski/morph/chess/agent.py", "app/agent.py")

In [96]:
# basically all file modifications here
# rmvd apt-get install -y python3
startup_commands = """
apt-get update
curl -LsSf https://astral.sh/uv/install.sh | bash
cd app
uv add pyproject.toml
"""
_ = instance.exec(command=startup_commands)

In [101]:
print(instance.exec(command=startup_commands))

exit_code=0 stdout="Hit:1 http://deb.debian.org/debian bookworm InRelease\nHit:2 http://deb.debian.org/debian bookworm-updates InRelease\nHit:3 http://deb.debian.org/debian-security bookworm-security InRelease\nReading package lists...\nReading package lists...\nBuilding dependency tree...\nReading state information...\npython3 is already the newest version (3.11.2-1+b1).\n0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.\nno checksums to verify\ninstalling to /root/.local/bin\n  uv\n  uvx\neverything's installed!\n" stderr='downloading uv 0.7.2 x86_64-unknown-linux-gnu\nUsing CPython 3.11.2 interpreter at: /usr/bin/python3\nCreating virtual environment at: .venv\nResolved 60 packages in 2.41s\n   Building chess==1.11.2\nDownloading cryptography (4.0MiB)\nDownloading pygments (1.2MiB)\nDownloading setuptools (1.1MiB)\nDownloading pygame (13.3MiB)\nDownloading pillow (4.4MiB)\nDownloading pydantic-core (1.9MiB)\n Downloading pygments\n Downloading setuptools\n Downloading p

In [103]:
print(instance.exec(command="ls -lah app"))

exit_code=0 stdout='total 272K\ndrwxr-xr-x 3 root root 4.0K May  5 14:40 .\ndrwx------ 7 root root 4.0K May  5 14:39 ..\ndrwxr-xr-x 5 root root 4.0K May  5 14:40 .venv\n-rw-r--r-- 1 root root  37K May  5 14:34 agent.py\n-rw-r--r-- 1 root root 6.2K May  5 14:34 chess_game.py\n-rw-r--r-- 1 root root  227 May  5 14:34 config.yaml\n-rw-r--r-- 1 root root  356 May  5 14:34 pyproject.toml\n-rw-r--r-- 1 root root 1.1K May  5 14:34 requirements.txt\n-rw-r--r-- 1 root root 1.1K May  5 14:34 schemas.py\n-rw-r--r-- 1 root root 196K May  5 14:40 uv.lock\n' stderr=''


In [100]:
print(instance.exec(command="uv run app/agent.py"))

exit_code=1 stdout='' stderr='Traceback (most recent call last):\n  File "/root/app/agent.py", line 7, in <module>\n    import yaml\nModuleNotFoundError: No module named \'yaml\'\n'


In [70]:
# branch from instance with files transferred + uv + deps installed
snapshot, clones = instance.branch(count=3)
print(f"Snapshot created: {snapshot.id}. This is the base snapshot for all chess branches.")

Snapshot created: snapshot_c6fyenbw. This is the base snapshot for all chess branches.


In [71]:
game_ids = [1,2,3]
strategies = ["aggressive", "defensive", "balanced"]
instance_startup = f"""
cd app
source .venv/bin/activate
export OPENAI_API_KEY="{os.environ.get("OPENAI_API_KEY")}"
"""

In [76]:
for clone, game_id, strategy in zip(clones, game_ids, strategies):
    print(f"Starting game {game_id} on instance {clone.id} with strategy {strategy}")
    #instance = client.instances.get(instance_id=clone.id)
    
    start_game_command = f"python agent.py --game-id {game_id} --moves 2"
    result = clone.exec(command=instance_startup + "\n" + start_game_command)
    print(f"Stdout:\n{result.stdout}")
    print(f"Stderr:\n{result.stderr}")

Starting game 1 on instance morphvm_3gq5x8sj with strategy aggressive
Stdout:
Using custom game ID: 1
Starting a new chess game...
Starting chess process with command: /root/app/.venv/bin/python3 ./chess_game.py --autosave --game-id 1 --strategy balanced
Chess process started, waiting for initial output...
Initial chess output received, parsing board state...
```json
{
  "selected_move": "e2e4",
  "alternative_moves": ["d2d4", "g1f3"]
}
```
Successfully parsed structured move: e2e4
LLM's move (balanced strategy): e2e4
Move 1 completed
```json
{
  "selected_move": "e7e5",
  "alternative_moves": ["c7c5", "g8f6"]
}
```
Successfully parsed structured move: e7e5
LLM's move (balanced strategy): e7e5
Move 2 completed
Game finished!
Game ended after 2 moves

Stderr:

Starting game 2 on instance morphvm_2rsghxfc with strategy defensive
Stdout:
Using custom game ID: 2
Starting a new chess game...
Starting chess process with command: /root/app/.venv/bin/python3 ./chess_game.py --autosave --game-i

In [82]:
print(clones[0].exec("echo \"hello?\" > tmp.txt").stdout)

In [77]:
# check game results on clone 1
#game_1_branch = client.instances.get(instance_id=clones[0].id)
print(clones[0].exec(f"cat app/chess_autosaves/game_id_{game_ids[0]}.json").stderr)
print("="*50)
print(clones[0].exec(f"cat app/chess_autosaves/game_id_{game_ids[0]}.json").stdout)

cat: app/chess_autosaves/game_id_1.json: No such file or directory




In [85]:
game_1_branch.id

'morphvm_3gq5x8sj'

In [84]:
print(game_1_branch.exec("ls").stdout)

app
tmp.txt



In [47]:
game_1_branch.exec(f"cat chess_autosaves/game_id_{game_ids[0]}.json")

InstanceExecResponse(exit_code=1, stdout='', stderr='cat: chess_autosaves/game_id_1.json: No such file or directory\n')

---

# utils


In [106]:
# delete all instances
instances = client.instances.list()

for instance in instances:
    print(f"Stopping instance ID: {instance.id}, Current Status: {instance.status}, Snapshot ID: {instance.refs.snapshot_id}")
    instance.stop()

In [107]:
# delete all snapshots
snapshots = client.snapshots.list()

for snapshot in snapshots:
    print(f"Deleting snapshot ID: {snapshot.id}, Created At: {snapshot.created}")
    snapshot.delete()

Deleting snapshot ID: snapshot_664am2xe, Created At: 1746455649
